In [16]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))
from analysis.jupyter_helper_functions import load_dataset, save_json, standardize_and_evaluate
from llm_data_quality_assistant.pipeline import Pipeline
from pprint import pprint
import json

In [17]:
corrupt_dataset = load_dataset(
    "../datasets/parker_datasets/flight/flight_cleaned_corrupted_first1000_int.csv"
)
gold_standard = load_dataset(
    "../datasets/parker_datasets/flight/flight_cleaned_gold_first1000_int.csv"
)
repaired_dataset = load_dataset(
    "../datasets/parker_datasets/flight/parker_repairs/flight_repair.csv"
)

# Check if all three dataframes have the same column order
same_column_order = (
    list(corrupt_dataset.columns) == list(gold_standard.columns) == list(repaired_dataset.columns)
)
print("All dataframes have the same column order:", same_column_order)

# Check if all three dataframes have the exact same row order
same_order = (
    (corrupt_dataset['composed_key'] == gold_standard['composed_key']).all() and
    (corrupt_dataset['composed_key'] == repaired_dataset['composed_key']).all()
)
print("All dataframes have the exact same row order:", same_order)

All dataframes have the same column order: False
All dataframes have the exact same row order: False


In [18]:
import time
start_time = time.time()

# Evaluate and save results using the helper function
standardize_and_evaluate(
    gold_standard=gold_standard,
    merged_df=repaired_dataset,
    corrupt_dataset=corrupt_dataset,
    primary_key="composed_key",
    time_delta=time.time() - start_time,
    results_dir="../analysis/results/flight/",
    file_name="parker"
)


{'accuracy': 0.3732991953182151,
 'column_names': ['composed_key',
                  'actual_arrival',
                  'actual_departure',
                  'scheduled_arrival',
                  'scheduled_departure'],
 'f1_score': 0.22214824006537331,
 'false_negative': 36453,
 'false_negative_rate': 0.7680298337652487,
 'false_positive': 40650,
 'false_positive_rate': 0.5379332248203581,
 'num_columns': 5,
 'num_rows': 24606,
 'precision': 0.21312427409988385,
 'recall': 0.23197016623475128,
 'time_taken': 5.245208740234375e-05,
 'true_negative': 34917,
 'true_positive': 11010}
{'column_names': ['composed_key',
                  'actual_arrival',
                  'actual_departure',
                  'scheduled_arrival',
                  'scheduled_departure'],
 'num_columns': 5,
 'num_rows': 24606,
 'stats': [{'accuracy': 1.0,
            'column_name': 'composed_key',
            'f1_score': 0.0,
            'false_negative': 0,
            'false_negative_rate': 0.0,
        